# CNN with Keras on CIFAR10

Modify your MLP version from the previous exercise towards Convolutional Neural Networks.

## Loading the packages

In [ ]:
# First, import TF and get its version.
import tensorflow as tf
tf_version = tf.__version__

# Check if version >=2.0.0 is used
if not tf_version.startswith('2.'):
    print('WARNING: TensorFlow >= 2.0.0 will be used in this course.\nYour version is {}'.format(tf_version) + '.\033[0m')
else:
    print('OK: TensorFlow >= 2.0.0' + '.\033[0m')

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation, Dropout
from tensorflow.keras import utils
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import confusion_matrix

%matplotlib inline

# fix random seed for reproducibility
np.random.seed(42)

# GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## Loading the raw data

In [ ]:
def show_imgs(X):
    plt.figure(1)
    k = 0
    for i in range(0,5):
        for j in range(0,5):
            plt.subplot2grid((5,5),(i,j))
            plt.imshow(X[k], cmap='gray')
            k = k+1
            plt.axis('off')
    # show the plot
    plt.show()
    
# Load data & split data between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
CLASS_NAMES = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

show_imgs(X_train)

In [ ]:
# Normalize input
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

n_classes = 10
# Conversion to class vectors
Y_train = utils.to_categorical(y_train, n_classes)
Y_test = utils.to_categorical(y_test, n_classes)

## Simple CNN

### Define the network

In [ ]:
E = 30
B = 128
D = X_train.shape[1:]

def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), input_shape=D, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2))
    model.add(Flatten())
    model.add(Dense(n_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])
    
    return model

tf.keras.backend.clear_session()
model = create_model()
model.summary()

### Compile and train the network

In [ ]:
log = model.fit(X_train, Y_train, batch_size=B, epochs=E,
                    verbose=1, validation_data=(X_test, Y_test))

### Evaluate the network

#### Loss evolution during training
This can be done first looking at the history of the training (output of the `fit()` function).

In [ ]:
f = plt.figure(figsize=(12,4))
ax1 = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax1.plot(log.history['loss'], label='Training loss')
ax1.plot(log.history['val_loss'], label='Testing loss')
ax1.legend()
ax1.grid()
ax2.plot(log.history['accuracy'], label='Training acc')
ax2.plot(log.history['val_accuracy'], label='Val acc')
ax2.legend()
ax2.grid()

#### Overall accuracy evaluation on test set
We can compute the overall performance on test set calling the `evaluate()` function on the model. The function returns the loss and the metrics used to compile the models.

In [ ]:
loss_train, metric_train = model.evaluate(X_train, Y_train, verbose=0)
loss_test, metric_test = model.evaluate(X_test, Y_test, verbose=0)
print('TRAIN: \t loss: {0:.4f}\t accuracy: {1:.4f}'.format(loss_train, metric_train))
print('TEST: \t loss: {0:.4f}\t accuracy: {1:.4f}'.format(loss_test, metric_test))

## Deeper CNN

In [ ]:
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

project_id = 'deep-learning-sw08'

### Define the network

In [ ]:
D = X_train.shape[1:]

# initialize wandb with your project name and optionally with configutations.
run = wandb.init(project=project_id,
           config={
              "learning_rate": 0.005,
              "epochs": 25,
              "batch_size": 64,
               "activation": 'relu',
              "loss_function": "categorical_crossentropy",
              "architecture": "CNN",
              "dataset": "CIFAR-10"
           })
config = wandb.config

def create_model(config):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), input_shape=D, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2))
    model.add(Flatten())
    model.add(Dense(n_classes, activation='softmax'))
    
    model.compile(loss=config.loss_function, 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate), 
                  metrics=['accuracy'])
    
    return model

tf.keras.backend.clear_session()
model = create_model(config)
model.summary()

### Compile and train the network
In Keras, we call the methods `compile()` and `fit()`. For the compile phase, we need to specify the **loss** function which should be set in the case of multi-class classification to `categorical_crossentropy`. We also need to specify the optimizer strategy. In this case the `rmsprop` or `adam` can be used.

In [ ]:
# in order to get prediction on small subset of images.
val_images, val_labels = X_test[:32], Y_test[:32]

log = model.fit(X_train, Y_train, 
                epochs=config.epochs, 
                batch_size=config.batch_size,
                validation_data=(X_test, Y_test),
                callbacks=[WandbCallback(data_type='image', 
                                   training_data=(val_images, val_labels), 
                                   labels=CLASS_NAMES)],
                verbose=0)

### Evaluate the network

#### Loss evolution during training
This can be done first looking at the history of the training (output of the `fit()` function).

In [ ]:
f = plt.figure(figsize=(12,4))
ax1 = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax1.plot(log.history['loss'], label='Training loss')
ax1.plot(log.history['val_loss'], label='Testing loss')
ax1.legend()
ax1.grid()
ax2.plot(log.history['accuracy'], label='Training acc')
ax2.plot(log.history['val_accuracy'], label='Val acc')
ax2.legend()
ax2.grid()

#### Overall accuracy evaluation on test set
We can compute the overall performance on test set calling the `evaluate()` function on the model. The function returns the loss and the metrics used to compile the models.

In [ ]:
loss_train, metric_train = model.evaluate(X_train, Y_train, verbose=0)
loss_test, metric_test = model.evaluate(X_test, Y_test, verbose=0)
print('TRAIN: \t loss: {0:.4f}\t accuracy: {1:.4f}'.format(loss_train, metric_train))
print('TEST: \t loss: {0:.4f}\t accuracy: {1:.4f}'.format(loss_test, metric_test))

# log to wandb
wandb.log({'Train Error Rate': round((1-metric_train)*100, 2)})
wandb.log({'Test Error Rate': round((1-metric_test)*100, 2)})

## Hyperparameter optimization

In [ ]:
def create_model(config):
    model = Sequential()
    
    ### 1. layer
    model.add(Conv2D(filters=config.filters[0], 
                     kernel_size=(config.kernel, config.kernel), 
                     strides=1, 
                     padding="same", 
                     activation=None,
                     kernel_initializer=config.initializer,
                     input_shape=(32,32,3)))
    model.add(BatchNormalization())
    model.add(Activation(config.activation))
    
    ### 2. layer
    model.add(Conv2D(filters=config.filters[0], 
                     kernel_size=(config.kernel, config.kernel), 
                     strides=1, 
                     padding="same", 
                     activation=None,
                     kernel_initializer=config.initializer))
    model.add(BatchNormalization())
    model.add(Activation(config.activation))
    
    ### 3.+ layers
    for layer in config.filters[1:]:        
        model.add(Conv2D(filters=layer, 
                         kernel_size=(config.kernel, config.kernel), 
                         strides=1, 
                         padding="same", 
                         activation=None,
                         kernel_initializer=config.initializer))
        model.add(BatchNormalization())
        model.add(Activation(config.activation))
        model.add(MaxPooling2D())
        model.add(Dropout(0.2))
    
    ### 2nd last layer
    model.add(Flatten())
    model.add(tf.keras.layers.Dense(1024, activation=None, kernel_initializer=config.initializer))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(config.activation))
    model.add(tf.keras.layers.Dropout(0.2))
    
    ### last layer
    model.add(Dense(n_classes, activation='softmax'))
    
    return model

In [ ]:
def train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'batch_size': 32,
        'learning_rate': 0.001,
        'filters': [32, 64, 128, 256, 512],
        'activation': 'relu',
        'initializer': 'glorot_uniform',
        'kernel': 3
    }
    
    # Initialize wandb with a sample project name
    wandb.init(project=project_id,
               config=config_defaults)

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 30
    wandb.config.loss_function = 'categorical_crossentropy'
    config = wandb.config
    
    # define model
    tf.keras.backend.clear_session()
    model = create_model(config)
    
    # compile model
    model.compile(loss=config.loss_function, 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate), 
                  metrics=['accuracy'])
    
    # train model
    _ = model.fit(X_train, Y_train, 
                epochs=config.epochs, 
                batch_size=config.batch_size,
                validation_data=(X_test, Y_test),
                callbacks=[WandbCallback()],
                verbose=0)
    
    # evaluate model
    loss_test, metric_test = model.evaluate(X_test, Y_test, verbose=0, callbacks=[WandbCallback()])
    wandb.log({'Test Error Rate': round((1-metric_test)*100, 2)})

In [ ]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [32, 64, 128, 256]
      },
      'learning_rate': {
          'values': [0.005, 0.001, 0.0005, 0.0001]
      },
      'filters': {
          'values': [
              [32, 64, 128, 256, 512],
              [32, 64, 128, 256, 512, 512],
          ]
      },
      'activation': {
          'values': ['elu', 'selu', 'relu']
      },
      'initializer': {
          'values': ['he_normal', 'he_uniform', 'glorot_uniform', 'glorot_normal', 'lecun_normal']
      },
      'kernel': {
          'values': [3]
      }
  }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project=project_id)
wandb.agent(sweep_id, function=train)

### Results

In [ ]:
df_results = pd.read_csv('wandb_export_sweep.csv')
df_results.sort_values('val_accuracy', ascending=False)

### Train Best Model

In [ ]:
config = wandb.config
config.activation = 'relu'
config.batch_size = 32
config.filters = [32,64,128,256,512,512]
config.initializer = 'he_normal'
config.learning_rate = 0.0005
config.kernel = 3
config.loss_function = 'categorical_crossentropy'
config.epochs = 30

tf.keras.backend.clear_session()
best_model = create_model(config)

# compile model
best_model.compile(loss=config.loss_function, 
                optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate), 
                metrics=['accuracy'])

# early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                                  patience=5,
                                                  restore_best_weights=True)

# train model
log = best_model.fit(X_train, Y_train, 
                    epochs=config.epochs, 
                    batch_size=config.batch_size,
                    validation_data=(X_test, Y_test),
                    callbacks=[early_stopping],
                    verbose=0)

# evaluate model
loss_train, metric_train = best_model.evaluate(X_train, Y_train, verbose=0)
loss_test, metric_test = best_model.evaluate(X_test, Y_test, verbose=0)
print('TRAIN: \t loss: {0:.4f}\t accuracy: {1:.4f}'.format(loss_train, metric_train))
print('TEST: \t loss: {0:.4f}\t accuracy: {1:.4f}'.format(loss_test, metric_test))

In [ ]:
f = plt.figure(figsize=(12,4))
ax1 = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax1.plot(log.history['loss'], label='Training loss')
ax1.plot(log.history['val_loss'], label='Testing loss')
ax1.legend()
ax1.grid()
ax2.plot(log.history['accuracy'], label='Training acc')
ax2.plot(log.history['val_accuracy'], label='Val acc')
ax2.legend()
ax2.grid()

**Best Model**   
TRAIN:
- loss: 0.0319
- accuracy: 0.9929   

TEST:
- loss: 0.4820
- accuracy: 0.8633